author: leezeeyee   
date: 2021/4/13  
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [1]:
import pandas as pd
import numpy as np

In [2]:
irisData=pd.read_csv('iris.csv')
irisData.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [3]:
## different types
setosas=irisData[irisData['Species']=='setosa']
versicolors=irisData[irisData['Species']=='versicolor']
virginicas=irisData[irisData['Species']=='virginica']

In [4]:
classes=3
Y=np.r_[np.zeros(len(setosas),dtype=int),np.ones(len(versicolors), dtype=int),2*np.ones(len(virginicas), dtype=int)]
one_hot_Y = np.eye(classes)[Y]


In [5]:
X=np.asarray(irisData[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']].iloc[:,:])

In [6]:
trainX=[]
trainY=[]
testX=[]
testY=[]

for i in range(classes):
    index=np.where(Y==i)[0]
    np.random.shuffle(index)
    
    trainX.append(X[index[:30]])
    trainY.append(one_hot_Y[index[:30]])
    testX.append(X[index[30:]])
    testY.append(one_hot_Y[index[30:]])
    print(index)
print(trainX[2][0])
print(X[index[0]])
trainX_all=np.r_[trainX[0],trainX[1],trainX[2]]
trainY_all=np.r_[trainY[0],trainY[1],trainY[2]]
indexTrain=np.arange(len(trainX_all))
np.random.shuffle(indexTrain)
trainX_sh=trainX_all[indexTrain]
trainY_sh=trainY_all[indexTrain]
testX_all=np.r_[testX[0],testX[1],testX[2]]
testY_all=np.r_[testY[0],testY[1],testY[2]]
indexTest=np.arange(len(testX_all))
np.random.shuffle(indexTest)
testX_sh=testX_all[indexTest]
testY_sh=testY_all[indexTest]

trainX_sh[:3]

[31 15 44 18 37 12 48  0  6 41 47 49 27 45 28 22 36 33 10 38  2  1 34 11
 14 23 13 24 26 42 25  5 35  7 40 30 19 16 21 20  3  8 17 39 29  4 46 32
  9 43]
[52 60 98 90 65 79 54 92 81 62 66 83 96 58 61 73 74 56 69 57 86 71 76 95
 85 51 77 84 50 67 82 75 72 55 94 78 64 63 93 89 87 59 99 68 80 91 53 70
 88 97]
[130 134 118 128 143 145 120 100 121 124 144 141 132 101 117 137 116 136
 127 138 106 126 148 129 131 104 114 142 110 140 149 113 105 125 119 122
 102 115 147 146 107 135 112 139 133 109 123 108 103 111]
[7.4 2.8 6.1 1.9]
[7.4 2.8 6.1 1.9]


array([[6.3, 3.4, 5.6, 2.4],
       [6.6, 2.9, 4.6, 1.3],
       [5. , 3.3, 1.4, 0.2]])

In [7]:
def softmax(X,Y,maxStep=120,stepSize=1):
    ## param
    # k for class num; N for dimension of data
    K=3
    N=4
    W=np.zeros([K,N])
    # M for the item num of data
    M=len(X)
    ## compute si
    softMaxY=np.zeros(K)
    # initialize W
    W=np.zeros([K,N])
    # record best W
    errMin=M
    for step in range(120):
        for i in range(M):
            # print(np.dot(W,np.transpose(X[i])))
            S=np.dot(W,np.transpose(X[i]))
            # print(np.dot(S,np.transpose(Y[i])))
            
            # print(np.where(y==1)[0][0]==np.where(S==np.max(S))[0][0])
            softMaxS=np.exp(S)
            sumY=np.sum(softMaxS)
            # print(sumY)
            # print(softMaxY)
            for j in range(K):
                softMaxY[j]=softMaxS[j]/sumY
                if Y[i][j]==1:
                    W[j]-=stepSize*(softMaxY[j]-1)*X[i]
                else:
                    W[j]-=stepSize*(softMaxY[j])*X[i]
            # print(step)
            # print(softMaxY)
            # print(W)

        Pass=True
        err=0
        for i in range(len(X)):
            y=Y[i]
            S=np.dot(W,np.transpose(X[i]))
            if np.where(y==1)[0][0] != np.where(S==np.max(S))[0][0]:
                err+=1
                Pass=False
        if Pass is True:
            print('success')
            break
        else:
            pass
        if err<errMin:
            W_end=W
            errMin=err
            print('err: '+str(err))
    return W_end

In [8]:
W_end=softmax(X=trainX_sh,Y=trainY_sh,maxStep=200,stepSize=0.1)

err: 30
err: 20
err: 13
err: 9
err: 8
err: 7
err: 4
err: 3


In [9]:
W_end

array([[ 3.42248318,  6.82203295, -8.79863451, -4.16972832],
       [ 4.53700006, -0.06791466, -1.93757125, -7.66808005],
       [-7.95948324, -6.75411828, 10.73620577, 11.83780837]])

## test

In [10]:
def testErr(X,Y):
    
    
    err=0
    for i in range(len(X)):
                y=Y[i]
                S=np.dot(W_end,np.transpose(X[i]))
                if np.where(y==1)[0][0] != np.where(S==np.max(S))[0][0]:
                    print(y)
                    print(S)
                    err+=1
    print(err)

In [11]:
testErr(X=testX_sh,Y=testY_sh)

[0. 0. 1.]
[-10.46429223   7.00920587   3.45508636]
[0. 1. 0.]
[-7.71580047  3.44808734  4.26771312]
[0. 0. 1.]
[-13.82493375   7.02110192   6.80383183]
3
